In [2]:
%pylab inline

%pylab is deprecated, use %matplotlib inline and import the required libraries.
Populating the interactive namespace from numpy and matplotlib


In [3]:
from zaber_motion import Units, Library, Measurement
from zaber_motion.ascii import Connection, AllAxes
from zaber_motion import *
import pandas as pd

In [4]:
Library.enable_device_db_store()
connection=Connection.open_serial_port("COM4")  # confirm that this is the right serial port

device_list = connection.detect_devices()
print("Found {} devices".format(len(device_list)))

device1 = device_list[0] 

Found 1 devices


In [5]:
# !python -m pip install -U zaber_motion

In [6]:
df_zlut=pd.read_excel('z_table.xlsx')
df_zlut.head()

NO_VELOCITY = True
# True=Let Zaber figure out what the velocities should be
# False=Use computed vels from xls. But the sign is important, unlike in xls.

step_size=5 # Can go through subsets of table with larger #s
idxs_subset=np.arange(0,50,step_size)
df_zlut=df_zlut.iloc[idxs_subset,:]

# Times are a period, not absolute time
# Assume they are equally spaced in time, and use the first one.
df_zlut.iloc[:,0]=np.diff( df_zlut.iloc[:,0] )[0]


# https://software.zaber.com/motion-library/docs/guides/pvt: 
# Make the velocity of the last one zero: necessary, it seems !

# Let Zaber figure out what the velocities should be
if NO_VELOCITY:
    df_zlut.iloc[:,1]=None
    df_zlut.iloc[0,1]=0
    df_zlut.iloc[-1,1]=0 #len(df_zlut.index),"Velocity (mm/s)"]=0

    #df_zlut.iloc[df_zlut["Displacement(mm)"]==0,1]=0
print( df_zlut)

    Time(s) Velocity (mm/s)  Degrees  Displacement(mm)
0       0.3               0    -38.4         19.467589
5       0.3            None    -30.4         12.373770
10      0.3            None    -22.4          6.790857
15      0.3            None    -14.4          2.827515
20      0.3            None     -6.4          0.560887
25      0.3            None      1.6          0.035090
30      0.3            None      9.6          1.260357
35      0.3            None     17.6          4.212840
40      0.3            None     25.6          8.835073
45      0.3               0     33.6         15.037088


In [8]:
ax1_sweep_lims=[0,10] # mm
ax3_sweep_lims=[10,0] # mm

In [12]:
#ax0=device1.get_axis(2)

pvt = device1.get_pvt(1)
pvt.disable()

pvt_buffer = device1.get_pvt_buffer(1)
pvt_buffer.erase()

NDIMS=3
arr_axes=np.arange(NDIMS)+1
# set up PVT to store points to PVT buffer 1 and
# to use the first axis for unit conversion
pvt.setup_store(pvt_buffer, *arr_axes )

# add PVT points from LUT in Excel file
for index, row in df_zlut.iterrows():

    # Add each point in a loop
    if NDIMS==1:
        poses=[Measurement(row["Displacement(mm)"], Units.LENGTH_MILLIMETRES)]
    elif NDIMS==2:
        ax1_pos = ax1_sweep_lims[0] + (index)*(ax1_sweep_lims[1]-ax1_sweep_lims[0])/df_zlut.index[-1]
        poses=[Measurement(ax1_pos, Units.LENGTH_MILLIMETRES),
               Measurement(row["Displacement(mm)"], Units.LENGTH_MILLIMETRES)]
    else: # Assume 3
        ax1_pos = ax1_sweep_lims[0] + (index)*(ax1_sweep_lims[1]-ax1_sweep_lims[0])/df_zlut.index[-1]
        ax3_pos = ax3_sweep_lims[0] + (index)*(ax3_sweep_lims[1]-ax3_sweep_lims[0])/df_zlut.index[-1]
        poses=[Measurement(ax1_pos, Units.LENGTH_MILLIMETRES),
               Measurement(row["Displacement(mm)"], Units.LENGTH_MILLIMETRES),
               Measurement(ax3_pos, Units.LENGTH_MILLIMETRES) ];
        
    if row["Velocity (mm/s)"] is None:
        vels=[None]*NDIMS
    else:
        vels=[Measurement(row["Velocity (mm/s)"], Units.VELOCITY_MILLIMETRES_PER_SECOND)]*NDIMS
           
    # Toggle the DIO (first channel): at 0 will be 1, at 1 will be 0, at 2 will be 1, etc..
    # use +1 mod 2. TODO (this is pretty hacky and inflexible)
    pvt.set_digital_output(1,(index+1)%2)
    pvt.point(poses, vels , Measurement(row["Time(s)"], Units.TIME_SECONDS) )
    #time.sleep(0.01)
pvt.set_digital_output(1,0)

# finish writing to the buffer
pvt.disable()

print(pvt_buffer)
print( pvt )
print( pvt_buffer.get_content() )

Device 1 -> PVT Sequence 1 (Disabled)
['setup store 1 3', 'io set do 1 1', 'point abs p 0 39242 20157 v 0 0 0 t 300.0', 'io set do 1 0', 'point abs p 2240 24942 17918 v 12232 -69777 -12232 t 300.0', 'io set do 1 1', 'point abs p 4479 13689 15678 v 12232 -52546 -12232 t 300.0', 'io set do 1 0', 'point abs p 6719 5700 13438 v 12232 -34292 -12232 t 300.0', 'io set do 1 1', 'point abs p 8959 1131 11199 v 12232 -15370 -12232 t 300.0', 'io set do 1 0', 'point abs p 11199 71 8959 v 12232 3850 -12232 t 300.0', 'io set do 1 1', 'point abs p 13438 2541 6719 v 12232 22996 -12232 t 300.0', 'io set do 1 0', 'point abs p 15678 8492 4479 v 12232 41694 -12232 t 300.0', 'io set do 1 1', 'point abs p 17918 17809 2240 v 12232 59580 -12232 t 300.0', 'io set do 1 0', 'point abs p 20157 30311 0 v 0 0 0 t 300.0', 'io set do 1 0', 'setup disable']


In [18]:
# execute:
pvt = device1.get_pvt(2)
pvt.disable()
pvt.setup_live(*arr_axes) # Execute on axis Z

# the sequence will now execute the stored points
pvt.call(pvt_buffer)

#print (pvt.

In [14]:
axs=[device1.get_axis(n) for n in [1,2,3]]

In [17]:
axs[0].move_absolute(0)
axs[1].move_absolute(40000)
axs[2].move_absolute(20000)

In [239]:
ax1.home()
ax2.home()

In [246]:
[ax1.get_position() for ax1 in axs]

[20157.0, 17809.0, 10.0]

In [169]:
ax1=device1.get_axis(1)
ax1.get_state()

'{"context":{"v":3,"type":"peripheral","fw":{"major":7,"minor":33,"build":14091}},"axis":{"peripheralId":"70131","serial":"112556","settings":{"cloop.continuous.enable":"0","cloop.displace.tolerance":"128","cloop.enable":"1","cloop.recovery.enable":"0","cloop.stall.action":"0","cloop.stall.detect.mode":"0","cloop.stall.tolerance":"512","cloop.timeout":"100","driver.current.hold":"35","driver.current.run":"70","ictrl.advance.a":"0.000000000","ictrl.advance.offset":"4096.000000000","ictrl.afcff.inductance":"0.088000000","ictrl.afcff.ke":"7.781229568","ictrl.afcff.ki":"0.000000000","ictrl.afcff.max":"48000.000851968","ictrl.afcff.ss":"0.000000000","ictrl.afcff.ss.max":"0.999899840","ictrl.ff.kd":"0.088000000","ictrl.ff.kp":"0.000999999","ictrl.gain.coldmult":"0.299999968","ictrl.pi.ki":"0.000028998","ictrl.pi.kp":"0.011596000","ictrl.type":"6","knob.dir":"0","knob.distance":"640","knob.enable":"1","knob.maxspeed":"153600","knob.mode":"0","knob.speedprofile":"2","limit.approach.maxspeed":"

In [177]:
ax3=device1.get_axis(3)
ax3.get_state()

print ()

'{"context":{"v":3,"type":"peripheral","fw":{"major":7,"minor":33,"build":14091}},"axis":{"peripheralId":"70134","serial":"112555","settings":{"cloop.continuous.enable":"0","cloop.displace.tolerance":"128","cloop.enable":"1","cloop.recovery.enable":"0","cloop.stall.action":"0","cloop.stall.detect.mode":"0","cloop.stall.tolerance":"512","cloop.timeout":"100","driver.current.hold":"35","driver.current.run":"70","ictrl.advance.a":"0.000000000","ictrl.advance.offset":"4096.000000000","ictrl.afcff.inductance":"0.088000000","ictrl.afcff.ke":"7.781229568","ictrl.afcff.ki":"0.000000000","ictrl.afcff.max":"48000.000851968","ictrl.afcff.ss":"0.000000000","ictrl.afcff.ss.max":"0.999899840","ictrl.ff.kd":"0.088000000","ictrl.ff.kp":"0.000999999","ictrl.gain.coldmult":"0.299999968","ictrl.pi.ki":"0.000028998","ictrl.pi.kp":"0.011596000","ictrl.type":"6","knob.dir":"0","knob.distance":"640","knob.enable":"1","knob.maxspeed":"153600","knob.mode":"0","knob.speedprofile":"2","limit.approach.maxspeed":"

In [178]:
ax3.home()

In [180]:
ax3.move_absolute(10)